In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from langchain.document_loaders import (
    GitLoader,
    YoutubeLoader,
    DataFrameLoader
)
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool

In [2]:
# Knowledge base
kb_loader = GitLoader(
    clone_url="https://github.com/neo4j-documentation/knowledge-base",
    repo_path="./repos/kb/",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".adoc")
    and "articles" in file_path,
)
kb_data = kb_loader.load()
print(len(kb_data))

309


In [3]:
# Define text chunk strategy
splitter = CharacterTextSplitter(
  chunk_size=2000, 
  chunk_overlap=50,
  separator=" "
)
# GDS guides
gds_loader = GitLoader(
    clone_url="https://github.com/neo4j/graph-data-science",
    repo_path="./repos/gds/",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".adoc") 
    and "pages" in file_path,
)
gds_data = gds_loader.load()
# Split documents into chunks
gds_data_split = splitter.split_documents(gds_data)
print(len(gds_data_split))

771


In [4]:
# Youtube
yt_loader = YoutubeLoader("1sRgsEKlUr0")
yt_data = yt_loader.load()
yt_data_split = splitter.split_documents(yt_data)
print(len(yt_data_split))

10


In [5]:
# Medium
article_url = "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/medium/neo4j_articles.csv"
medium = pd.read_csv(article_url, sep=";")
medium["source"] = medium["url"]
medium_loader = DataFrameLoader(
    medium[["text", "source"]], 
    page_content_column="text")
medium_data = medium_loader.load()
medium_data_split = splitter.split_documents(medium_data)
print(len(medium_data_split))

2244


In [6]:
# Stackoverflow
so_data = []
for i in range(1, 20):
    # Define the Stack Overflow API endpoint and parameters
    api_url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order": "desc",
        "sort": "creation",
        "filter": "!-MBrU_IzpJ5H-AG6Bbzy.X-BYQe(2v-.J",
        "tagged": "neo4j",
        "site": "stackoverflow",
        "pagesize": 100,
        "page": i,
    }
    # Send GET request to Stack Overflow API
    response = requests.get(api_url, params=params)
    data = response.json()
    # Retrieve the resolved questions
    resolved_questions = [
        question
        for question in data["items"]
        if question["is_answered"] and question.get("accepted_answer_id")
    ]

    # Print the resolved questions
    for question in resolved_questions:
        text = (
            "Title:",
            question["title"] + "\n" + "Question:",
            BeautifulSoup(question["body"]).get_text()
            + "\n"
            + BeautifulSoup(
                [x["body"] for x in question["answers"] if x["is_accepted"]][0]
            ).get_text(),
        )
        source = question["link"]
        so_data.append(Document(page_content=str(text), metadata={"source": source}))
print(len(so_data))


775


In [7]:
# Define embedding model
OPENAI_API_KEY = "OPENAI_API_KEY"
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

sales_data = medium_data_split + yt_data_split
sales_store = Chroma.from_documents(
    sales_data, embeddings, collection_name="sales"
)

support_data = kb_data + gds_data_split + so_data
support_store = Chroma.from_documents(
    support_data, embeddings, collection_name="support"
)

Using embedded DuckDB without persistence: data will be transient
Using embedded DuckDB without persistence: data will be transient


In [8]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    max_tokens=512,
)

In [9]:
sales_template = """As a Neo4j marketing bot, your goal is to provide accurate and helpful information about Neo4j,
a powerful graph database used for building various applications.
You should answer user inquiries based on the context provided and avoid making up answers.
If you don't know the answer, simply state that you don't know.
Remember to provide relevant information about Neo4j's features, benefits,
and use cases to assist the user in understanding its value for application development.

{context}

Question: {question}"""
SALES_PROMPT = PromptTemplate(
    template=sales_template, input_variables=["context", "question"]
)
sales_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sales_store.as_retriever(),
    chain_type_kwargs={"prompt": SALES_PROMPT},
)

In [10]:
print(sales_qa.run("What are the main benefits of using Neo4j?"))

Neo4j is a powerful graph database that offers several benefits for application development. Some of the main benefits include:

1. High Performance: Neo4j is designed to handle complex queries and large datasets, making it ideal for applications that require high performance.

2. Flexible Data Model: Neo4j's flexible data model allows developers to easily store and query complex relationships between data points, making it ideal for applications that require complex data modeling.

3. Scalability: Neo4j is highly scalable, allowing developers to easily add new nodes and relationships as their application grows.

4. Easy to Use: Neo4j's query language, Cypher, is easy to learn and use, making it accessible to developers of all skill levels.

5. Real-Time Insights: Neo4j's ability to quickly analyze complex relationships between data points allows developers to gain real-time insights into their data, making it ideal for applications that require real-time data analysis.

Overall, Neo4j

In [11]:
support_template = """
As a Neo4j Customer Support bot, you are here to assist with any issues 
a user might be facing with their graph database implementation and Cypher statements.
Please provide as much detail as possible about the problem, how to solve it, and steps a user should take to fix it.
If the provided context doesn't provide enough information, you are allowed to use your knowledge and experience to offer you the best possible assistance.

{context}

Question: {question}"""

SUPPORT_PROMPT = PromptTemplate(
    template=support_template, input_variables=["context", "question"]
)

support_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=support_store.as_retriever(),
    chain_type_kwargs={"prompt": SUPPORT_PROMPT},
)


In [12]:
print(support_qa.run("""
I am having my graph in a VM and i want to use GDS Plugins in my graph.
I didn't see any proper documentation to install it in my server where it was working only on locally.
Anyone clear me this to install the GDS Plugin in the VM ?
"""))

To install the GDS plugin on a Neo4j server running on a VM, you can follow the manual installation steps provided in the Neo4j documentation. 

1. Download the `neo4j-graph-data-science-[version].zip` file from the Neo4j Download Center.
2. Unzip the archive and move the `neo4j-graph-data-science-[version].jar` file into the `$NEO4J_HOME/plugins` directory.
3. Add the following to your `$NEO4J_HOME/conf/neo4j.conf` file:
```
dbms.security.procedures.unrestricted=gds.*
```
4. Check if the procedure allowlist is enabled in the `$NEO4J_HOME/conf/neo4j.conf` file and add the GDS library if necessary:
```
dbms.security.procedures.allowlist=gds.*
```
5. Restart Neo4j.

After following these steps, you should be able to use the GDS plugin on your Neo4j server running on the VM. You can verify the installation by running the `gds.version()` function in Neo4j Browser.


In [13]:
# the zero-shot-react-description agent will use the "description" string to select tools
tools = [
    Tool(
        name = "sales",
        func=sales_qa.run,
        description="""useful for when a user is interested in various Neo4j information, 
                       use-cases, or applications. A user is not asking for any debugging, but is only
                       interested in general advice for integrating and using Neo4j.
                       Input should be a fully formed question."""
    ),
    Tool(
        name = "support",
        func=support_qa.run,
        description="""useful for when when a user asks to optimize or debug a Cypher statement or needs
                       specific instructions how to accomplish a specified task. 
                       Input should be a fully formed question."""
    ),
]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [15]:
agent.run("""What are some GPT-4 applications with Neo4j?""")



> Entering new AgentExecutor chain...
This question is asking for general advice on using Neo4j with GPT-4, so the best tool to use would be sales.
Action: sales
Action Input: "What are some use-cases for integrating Neo4j with GPT-4?"
Observation: There are several use cases for integrating Neo4j with GPT-4. One example is using GPT-4 as a domain expert to help extract knowledge from a video transcript and then storing that information in a Neo4j knowledge graph. Another example is using GPT-4 for relationship extraction to accelerate knowledge graph construction in Neo4j. Additionally, GPT-4 can be used in conjunction with Neo4j to develop a knowledge graph-based chatbot that provides answers based on data stored in the knowledge graph. Overall, integrating Neo4j with GPT-4 can enhance the capabilities of both technologies and enable more powerful and accurate applications.
Thought:I now know the final answer
Final Answer: There are several use cases for integrating Neo4j with GPT-

'There are several use cases for integrating Neo4j with GPT-4, including using GPT-4 as a domain expert to extract knowledge from a video transcript and storing it in a Neo4j knowledge graph, using GPT-4 for relationship extraction to accelerate knowledge graph construction, and developing a knowledge graph-based chatbot that provides answers based on data stored in the knowledge graph.'

In [19]:
agent.run("""
Hello everyone, is there a way to execute a weighted shortest path query in the Neo4j Community edition?
All I have found on the internet was gds library or the algo library, which are both unavailable the community version.
""")



> Entering new AgentExecutor chain...
This sounds like a technical question that requires specific instructions or optimization. 

Action: support
Action Input: Ask the user for more information about their specific use case and what they are trying to accomplish with the weighted shortest path query. Ask if they have tried any alternative methods or libraries. 
Observation: As a Neo4j Customer Support bot, I would like to ask for more information about your specific use case and what you are trying to accomplish with the weighted shortest path query. Have you tried any alternative methods or libraries? This will help me provide you with the best possible assistance.
Thought:Based on the user's response, I may need to provide specific instructions or suggest alternative methods.

Action: support
Action Input: Based on the user's response, provide specific instructions or suggest alternative methods for executing a weighted shortest path query in the Neo4j Community edition.
Observati

"To execute a weighted shortest path query in the Neo4j Community edition, you can use the APOC library or implement Dijkstra's algorithm manually in Cypher. The APOC library provides a built-in function for Dijkstra's algorithm, while the manual implementation requires a Cypher query that finds the shortest path based on the sum of the weights of the relationships in the path."